In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam

In [ ]:
!unzip '/content/drive/My Drive/rethink_ux_text_recognition_hiring_challenge-dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: TRAIN/038478.jpg        
  inflating: TRAIN/021371.jpg        
  inflating: TRAIN/066824.jpg        
  inflating: TRAIN/137220.jpg        
  inflating: TRAIN/052749.jpg        
  inflating: TRAIN/082471.jpg        
  inflating: TRAIN/136881.jpg        
  inflating: TRAIN/103260.jpg        
  inflating: TRAIN/150822.jpg        
  inflating: TRAIN/160994.jpg        
  inflating: TRAIN/015188.jpg        
  inflating: TRAIN/021501.jpg        
  inflating: TRAIN/115087.jpg        
  inflating: TRAIN/018680.jpg        
  inflating: TRAIN/032707.jpg        
  inflating: TRAIN/063475.jpg        
  inflating: TRAIN/003334.jpg        
  inflating: TRAIN/082355.jpg        
  inflating: TRAIN/069979.jpg        
  inflating: TRAIN/132503.jpg        
  inflating: TRAIN/060377.jpg        
  inflating: TRAIN/020671.jpg        
  inflating: TRAIN/049302.jpg        
  inflating: TRAIN/145852.jpg        
  inflating: TRAIN/061161.jpg        

In [ ]:
df = pd.read_csv('TRAIN.csv')
df

FILENAME     VALUES
0      000000.jpg   KESSELER
1      000001.jpg      LUCAS
2      000002.jpg       ELYN
3      000003.jpg     ALONZO
4      000004.jpg       NINO
...           ...        ...
79995  079995.jpg     ARTHUR
79996  079996.jpg      FIONA
79997  079997.jpg  DUFRENOIS
79998  079998.jpg  MADELEINE
79999  079999.jpg       AMAN

[80000 rows x 2 columns]

In [ ]:
print("Number of NaNs in train set      : ", df['VALUES'].isnull().sum())
df.dropna(axis=0, inplace=True)

df = df[df['VALUES'] != 'UNREADABLE']
df['VALUES'] = df['VALUES'].str.upper()
df.reset_index(inplace = True, drop=True) 

df['FILENAME'] = df['FILENAME'].str.rstrip('.jpg')
df

Number of NaNs in train set      :  131


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


FILENAME     VALUES
0       000000   KESSELER
1       000001      LUCAS
2       000002       ELYN
3       000003     ALONZO
4       000004       NINO
...        ...        ...
79842   079995     ARTHUR
79843   079996      FIONA
79844   079997  DUFRENOIS
79845   079998  MADELEINE
79846   079999       AMAN

[79847 rows x 2 columns]

In [ ]:
def preprocess(img):
    (h, w) = img.shape
    
    final_img = np.ones([64, 256])*255 # blank white image
    
    # crop
    if w > 256:
        img = img[:, :256]
        
    if h > 64:
        img = img[:64, :]
    
    
    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)

In [ ]:
train_x = []
for i in range(83600,123600):
    img_dir = 'TRAIN/'+df.loc[i, 'FILENAME']+'.jpg'
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    train_x.append(image)

In [ ]:
valid_x = []

for i in range(70000,77000):
    img_dir = 'TRAIN/'+df1.loc[i, 'FILENAME']+'.jpg'
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    valid_x.append(image)

In [ ]:
len(train_x)

40000

In [ ]:
train_x = np.array(train_x).reshape(-1, 256, 64, 1)
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)

In [ ]:
alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = 24 # max length of input labels
num_of_characters = len(alphabets) + 1 # +1 for ctc pseudo blank
num_of_timestamps = 64 # max length of predicted labels


def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))
        
    return np.array(label_num)

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

In [ ]:
train_y = np.ones([40000, max_str_len]) * -1
train_label_len = np.zeros([40000, 1])
train_input_len = np.ones([40000, 1]) * (num_of_timestamps-2)
train_output = np.zeros([40000])

for i in range(40000):
    train_label_len[i] = len(df.loc[i+83600, 'VALUES'])
    train_y[i, 0:len(df.loc[i+83600, 'VALUES'])]= label_to_num(df.loc[i+83600, 'VALUES'])  

In [ ]:
valid_y = np.ones([4000, max_str_len]) * -1
valid_label_len = np.zeros([4000, 1])
valid_input_len = np.ones([4000, 1]) * (num_of_timestamps-2)
valid_output = np.zeros([4000])

for i in range(4000):
    valid_label_len[i] = len(df.loc[i+123600, 'VALUES'])
    valid_y[i, 0:len(df.loc[i+123600, 'VALUES'])]= label_to_num(df.loc[i+123600, 'VALUES']) 

In [ ]:
input_data = Input(shape=(256, 64, 1), name='input')

inner = Conv2D(32, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)  
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)

inner = Conv2D(64, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout(0.3)(inner)

inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
inner = Dropout(0.3)(inner)

# CNN to RNN
inner = Reshape(target_shape=((64, 1024)), name='reshape')(inner)
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)

## RNN
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm1')(inner)
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters, kernel_initializer='he_normal',name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)
from keras.models import load_model
model = load_model('model1 (1).h5', compile=False)
model = Model(inputs=input_data, outputs=y_pred)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 64, 1)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 64, 32)       320       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 64, 32)       128       
_________________________________________________________________
activation (Activation)      (None, 256, 64, 32)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 128, 32, 32)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 128, 32, 64)       18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 32, 64)      

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [ ]:
# the loss calculation occurs elsewhere, so we use a dummy lambda function for the loss
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(lr = 0.0001))

model_final.fit(x=[train_x, train_y, train_input_len, train_label_len], y=train_output, 
                validation_data=([valid_x, valid_y, valid_input_len, valid_label_len], valid_output),
                epochs=60, batch_size=128)

Epoch 1/60
313/313 [==============================] - ETA: 0s - loss: 24.2227

InvalidArgumentError: ignored

In [ ]:
preds = model.predict(valid_x)
decoded = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

prediction = []
for i in range(4000):
    prediction.append(num_to_label(decoded[i]))

In [ ]:
decoded

array([[ 2, 11, 14, ..., -1, -1, -1],
       [ 0,  3,  0, ..., -1, -1, -1],
       [ 2, 11, 14, ..., -1, -1, -1],
       ...,
       [11, 14,  7, ..., -1, -1, -1],
       [ 2,  7, 14, ..., -1, -1, -1],
       [ 6,  0, 17, ..., -1, -1, -1]])

In [ ]:
y_true = df.loc[83600:123599, 'VALUES']
correct_char = 0
total_char = 0
correct = 0
y_true

80000             CLOAREC
80001               ADAMO
80002                CLOE
80003            DELAMARE
80004    VALENTIN-LAMBERT
               ...       
83595         DAMOTASOUSA
83596         PLUSQUELLEC
83597               LOHAN
83598         CHOUANGMALA
83599             GARNIER
Name: VALUES, Length: 3600, dtype: object

In [ ]:
y_true[83600]

'CLOAREC'

In [ ]:
for i in range(4000):
    pr = prediction[i]
    tr = y_true[i+123600]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/3600))

Correct characters predicted : 88.39%
Correct words predicted      : 75.58%


In [ ]:
df2 = pd.read_csv('TRAIN.csv')
df1= pd.DataFrame()
df1['FILENAME'] = df2.FILENAME[0:20685]
df1

FILENAME
0      000000.jpg
1      000001.jpg
2      000002.jpg
3      000003.jpg
4      000004.jpg
...           ...
20680  020680.jpg
20681  020681.jpg
20682  020682.jpg
20683  020683.jpg
20684  020684.jpg

[20685 rows x 1 columns]

In [ ]:
df1['FILENAME'] = df1['FILENAME'].str.rstrip('.jpg')
df1

FILENAME
0       000000
1       000001
2       000002
3       000003
4       000004
...        ...
20680   020680
20681   020681
20682   020682
20683   020683
20684   020684

[20685 rows x 1 columns]

In [ ]:
pred1=[]
decoded1=[]
for i in range(20685):
    img_dir = 'TEST/'+df1.loc[i, 'FILENAME']+".jpg"
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    pred = model.predict(image.reshape(1, 256, 64, 1))
    decoded = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], 
                                       greedy=True)[0][0])
    decoded1.append(decoded)

In [ ]:
decoded1

[array([[ 6,  0, 18, 18,  0, 13,  3, 17,  4, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]),
 array([[15,  4, 19,  0, 20, 24, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]),
 array([[ 1, 14, 13, 13,  8, 13, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]),
 array([[19,  0, 13, 16, 20,  4, 17,  4, 11, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -

In [ ]:
for i in range(20685):
  prediction1.append(num_to_label(decoded1[i][0]))

In [ ]:
df3=pd.DataFrame()
df3['VALUES']=prediction1
df3

VALUES
0      GASSANDRE
1         PETAUY
2         BONNIN
3      TANQUEREL
4           COZE
...          ...
20680     ALEXYS
20681  ALEXANDRE
20682        LGA
20683      JANHE
20684        LIM

[20685 rows x 1 columns]

In [ ]:
model.save("model1.h5")

In [ ]:
df3.to_csv("submit.csv")